<a href="https://colab.research.google.com/github/nekoniii3/openai_3syu/blob/main/Colab_Note/Whisper_large_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Whisper large-v3ダウンロード版（APIキー不要）**
## **①～③を順番に実行してください<br>※最初に上部のランタイムからGPU（T4GPUなど）に変更して下さい**

# **①whisperなどインストール**

In [ ]:
!pip install gradio==4.19.2
!pip install openai-whisper==20231117

# **②large-v3モデルダウンロード（時間がかかります）**

In [ ]:
import whisper

model_size = "large-v3"

model = whisper.load_model(model_size)

# **③アプリ起動**

In [ ]:
import os
import gradio as gr

audio_format = [".mp3", ".mp4", ".mpeg", ".mpga", ".m4a", ".wav", ".webm"
                ,".MP3", ".MP4", ".MPEG", ".MPGA", ".M4A", ".WAV", ".WEBM"]

lang_code = {'Japanese': "ja", 'English': "en"}
mode_code = {'文字': "transcribe", '翻訳': "translate"}

max_file_size = 25 * 1024 * 1024

def set_state(openai_key, lang, mode, state):

    state["openai_key"]= openai_key
    state["lang"] = lang_code[lang]
    state["mode"] = mode_code[mode[0:2]]

    return state

def create_textfile(voice_msg, up_file ,state):

    # ファイル入力チェック
    if up_file is None:

        err_msg = "ファイルをアップしてください。"

        return None, None, "", None, err_msg

    # ファイルパスなど取得
    file_name = os.path.splitext(os.path.basename(up_file.name))[0]
    root, ext = os.path.splitext(up_file.name)

    if ext not in audio_format:

        # ファイル形式チェック
        err_msg = "指定した形式のファイルをアップしてください。（mp3, mp4, mpeg, mpga, m4a, wav, webm）"

        return None, None, "", None, err_msg

    file_size = os.path.getsize(up_file.name)

    if file_size >= max_file_size:

        # ファイルサイズチェック
        err_msg = "ファイルが大きすぎます。25MB未満にして下さい。"

        return None, up_file.name, "", None, err_msg

    try:

        result = model.transcribe(up_file.name, language=state["lang"])

        trans_text = result["text"]

        # ファイル名設定
        trans_file = file_name + "_whisper.txt"

        with open(trans_file, mode="w") as f:

            # テキストに書き出す
            f.write(trans_text)

        return None, None, trans_text, trans_file, ""

    except Exception as e:

        return voice_msg, None, "", None, e


with gr.Blocks() as demo:

    title = "<h2>Whisperデモアプリ</h2>"

    # セッションの宣言
    state = gr.State({
        "openai_key" : "",
        "lang": "Japanese",
        "mode" : "",
    })

    with gr.Tab("whisperを利用する") as maintab:

      # 各コンポーネント定義
      voice_msg=gr.components.Audio(sources="upload",type="filepath", label="音声入力", visible= False)
      up_file = gr.File(file_types=[".mp3", ".mp4", ".mpeg", ".mpga", ".m4a", ".wav", ".webm"], label="音声ファイルアップロード")

      # ボタン類
      with gr.Row():
        btn = gr.Button("文字に起こす")
        clear = gr.ClearButton([voice_msg, up_file], value="クリア")

      # 出力
      sys_msg = gr.Text(label="システムメッセージ")
      text = gr.TextArea(label="文字起こし内容")
      file = gr.File(label="出力テキストファイル")

      # 送信ボタンクリック時の処理
      btn.click(create_textfile, inputs=[voice_msg, up_file, state], outputs=[voice_msg, up_file, text, file, sys_msg], queue=False)

    with gr.Tab("設定") as settab:
      openai_key = gr.Textbox(label="OpenAI API Key", interactive = True, visible= False)
      lang = gr.Dropdown(label="Language", choices=["Japanese", "English"], value = "Japanese", interactive = True)
      mode = gr.Dropdown(label="Mode", choices=["文字起こし", "翻訳＋文字起こし"], value = "文字起こし", interactive = True, visible= False)

      # 設定変更時
      maintab.select(set_state, [openai_key, lang, mode, state], state)

    with gr.Tab("利用上の注意"):

      caution = "利用上の注意<br>"
      caution += "・文字に起こせるファイル形式は mp3, mp4, mpeg, mpga, m4a, wav, webm のみです。<br>"
      caution += "・ファイルのサイズ上限は25MBです。<br>"
      # caution += "・翻訳は日本語の音声を英語のテキストに変えます。英語の音声→日本語はできません。<br>"
      caution += "<br>免責事項<br>本アプリはOpenAIのAPIで製作されており、利用で生じた損害について一切の責任を負えません。"
      gr.Markdown("<h3>" + caution + "</h3>")

demo.queue()
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9af389bce2d6e7e6dd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9af389bce2d6e7e6dd.gradio.live
